## Load dependencies and load data


In [1]:
#Loading dependencies
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


#Geo map of Cases per country globally
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

app =  dash.Dash(suppress_callback_exceptions = True)
server =  app.server

app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'})

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
#Reading in data
url = 'https://api.covid19api.com/summary'
r = requests.get(url)
dictr = r.json()
header = dictr['Countries']
df = json_normalize(header)
df = df.drop(df.index[[0,224]])
df = df.drop(columns = 'Slug')
df = df.sort_values(by=['TotalConfirmed'], ascending=False)
df.head()

/home/tshepo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:49: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
235,United States of America,US,32454,731602,1889,38647,6295,64840,2020-04-19T07:25:51Z
208,Spain,ES,887,191726,41,20043,0,74797,2020-04-19T07:25:51Z
108,Italy,IT,3491,175925,482,23227,2200,44927,2020-04-19T07:25:51Z
74,France,FR,19,149149,642,19345,1581,36587,2020-04-19T07:25:51Z
81,Germany,DE,1945,143342,107,4459,2286,85400,2020-04-19T07:25:51Z


In [2]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()

#Separating columns

#Corona  cases: Total and New
df_tot_cases = df.loc[:,['Country','TotalConfirmed']].sort_values(by='TotalConfirmed', ascending=False)
df_new_cases = df.loc[:,['Country','NewConfirmed']]

#Death Cases: Total and New 
df_tot_death = df.loc[:,['Country','TotalDeaths']]
df_new_death = df.loc[:,['Country','NewDeaths']]

#Recovery cases: Total and new
df_tot_recover = df.loc[:,['Country','TotalRecovered']]
df_new_recover = df.loc[:,['Country','NewRecovered']]
                      
#Sorting cases by total                     
df_cases = df_tot_cases.merge(df_new_cases, on='Country').sort_values(by=['TotalConfirmed'],
                                                                      ascending=False)
df_death = df_tot_death.merge(df_new_death, on='Country').sort_values(by=['TotalDeaths'],
                                                                      ascending=False)
df_recover = df_tot_recover.merge(df_new_recover, on='Country').sort_values(by=['TotalRecovered'], 
                                                                            ascending=False)
# Print statement to be turned to buttons
print(color.BOLD + 'Glabal Covid 19 Virus statistics as at: '
      + color.END,df.Date[1])
print(color.BOLD + color.UNDERLINE +'Total Confirmed Cases: ' 
      + color.END ,'\n','Number of cases', tot_cases)
print(color.BOLD + color.UNDERLINE +'Recoveries World wide: ' 
      + color.END ,'\n','Number of cases', tot_recover)
print(color.BOLD + color.UNDERLINE +'Total Deaths: ' 
      + color.END ,'\n','Number of cases', tot_deaths)

Glabal Covid 19 Virus statistics as at:  2020-04-19T07:25:51Z
Total Confirmed Cases:  
 Number of cases 2316388
Recoveries World wide:  
 Number of cases 591653
Total Deaths:  
 Number of cases 159478


In [3]:
df_tot_cases

,Country,TotalConfirmed
235,United States of America,731602
208,Spain,191726
108,Italy,175925
74,France,149149
81,Germany,143342
...,...,...
77,French Southern Territories,0
206,South Georgia and the South Sandwich Islands,0
106,Isle of Man,0
175,Pitcairn,0


In [4]:
df['Active Cases'] = df['TotalConfirmed'] - df['TotalRecovered'] - df['TotalDeaths']
df['Closed Cases'] = df['TotalRecovered'] + df['TotalDeaths']
df['Recovery Rate'] = (df['TotalRecovered'] / df['TotalConfirmed'])*100
df['Case Fatality Rate'] = (df['TotalDeaths'] / df['TotalConfirmed']) * 100
df_act = df.loc[:,['Country','Active Cases']].sort_values(by=['Active Cases'], ascending=False)
df_clos = df.loc[:,['Country','Closed Cases']].sort_values(by=['Closed Cases'], ascending=False)
df_fat = df.loc[:,['Country','Case Fatality Rate']].sort_values(by=['Case Fatality Rate'], ascending=False)
df_rec= df.loc[:,['Country','Recovery Rate']].sort_values(by=['Recovery Rate'], ascending=False)

In [5]:
#Geo map of Cases per country globally
# df['text'] = df['Country'] + '<br>' + \
#     'Confirmed Cases ' + df['NewConfirmed'] + ' Confirmed Cases ' + df['TotalConfirmed'] + '<br>' + \
#     'Deaths ' + df['TotalDeath'] + ' Recoveries ' + df['TotalRecovered'] 

#Total confirmed world map
data = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['TotalConfirmed'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(155,155,155)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Number of cases'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig = go.Figure(data = data, layout = layout)

# Active cases world map

data1 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['Active Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(255,255,255)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Active cases'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig1 = go.Figure(data = data1, layout = layout)

#Closed cases world map
data2 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Closed Cases'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(245, 197, 66)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Closed Cases'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig2 = go.Figure(data = data2, layout = layout)

#Recovery Rate World Map

data3 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Recovery Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(455,455,455)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Recovery Rate (%)'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig3 = go.Figure(data = data3, layout = layout)


#Case fatality rate world map

data4 = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z =df['Case Fatality Rate'],
 locationmode = 'country names',
 marker = dict(
 line = dict (
 color = 'rgb(555,555,555)',
 width = 4,
 )  ),
 colorbar = dict(
 title = 'Case Fatality Rate (%)'
 )
 ) ]
layout = dict(
 autosize=True,
    
)
fig4 = go.Figure(data = data4, layout = layout)







In [6]:

# url1 = 'https://api.covid19api.com/live/country/south-africa/status/confirmed/date/2020-03-21T13:13:30Z'
# df1 = pd.read_json(url1, orient='columns')
# url2 = 'https://api.covid19api.com/country/south-africa/status/confirmed'
# df2 = pd.read_json(url2, orient='columns')
# url3 = 'https://api.covid19api.com/country/south-africa/status/confirmed/live'
# df3 = pd.read_json(url3, orient='columns')



# #


In [7]:
# url4 = 'https://covid19.soficoop.com/country/za'



In [8]:
# url5 = 'https://api.covid19api.com/total/dayone/country/south-africa'
# df5 = pd.read_json(url5, orient='columns')
# url6 = 'https://api.covid19api.com/all?from=2020-04-01T00:00:00Z&to=2020-12-31T00:00:00Z'
# df6 = pd.read_json(url6, orient='columns')


In [9]:

# df6

In [10]:
# df_rsa = df6[df6.Country == 'South Africa'].reset_index()
# df_rsa

In [11]:

# df4 = requests.get(url4).json()
# df4= pd.json_normalize(df4,record_path ='snapshots')
# df4

In [12]:
# df5 = pd.read_csv('https://raw.githubusercontent.com/dsfsi/covid19za/master/data/covid19za_provincial_cumulative_timeline_confirmed.csv')
# df5 = df5.fillna(0)
# df5 = df5.drop(['YYYYMMDD'],axis=1)
# df5

In [13]:
# Deaths = go.Scatter(
#     x=df2.Date,
#     y=df1.Deaths
# )
# Cases = go.Scatter(
#     x=df2.Date, 
#     y=df2.Cases
# )

# data = [Cases]
# layout1 = go.Layout(
#     yaxis=dict(
#         domain=[0, 0.33]
#     ),
#     legend=dict(
#         traceorder="reversed"
#     ),
#     yaxis2=dict(
#         domain=[0.33, 0.66]
#     ),
#     yaxis3=dict(
#         domain=[0.66, 1]
#     )
# )
# fig = go.Figure(data=data1, layout=layout1)
# fig.show()

In [14]:
# #ACtive cases in South Africa
# fig_line = go.Figure()
# fig_line.add_trace(go.Scatter(x=df_rsa.Date, y=df_rsa.Confirmed, name='Total Cases in SA'))
# fig_line.add_trace(go.Scatter(x = df4.timestamp,y= df4.active, name = 'Active Cases in SA'))
# # fig_line.add_trace(go.Scatter(x = df4.timestamp,y= df4.recovered))
# # fig_line.add_trace(go.Scatter(x = df4.timestamp,y= df4.deaths))
# fig_line.show()

In [15]:
# #Closed cases in South Africa
# fig_lin = go.Figure()

# fig_lin.add_trace(go.Scatter(x = df4.timestamp,y= df4.recovered, name = 'Total Recovered'))
# fig_lin.add_trace(go.Scatter(x = df4.timestamp,y= df4.deaths,name = 'Total Deaths'))
# fig_lin.show()

In [16]:
# #Closed cases in the world
# fig_lin = go.Figure()

# fig_lin.add_trace(go.Scatter(x = df4.timestamp,y= df4.recovered, name = 'Total Recovered'))
# fig_lin.add_trace(go.Scatter(x = df4.timestamp,y= df4.deaths,name = 'Total Deaths'))
# fig_lin.show()

In [18]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
#Rename Columns for readability
df = df.rename(columns={'NewConfirmed':'New Cases','TotalConfirmed':'Confirmed Cases',
                        'NewDeaths':'New Deaths', 'Total Deaths':'Confirmed Deaths',
                        'NewRecovered':'New Recoveries','TotalRecovered':'Confirmed Recoveries'})

#App
app = dash.Dash(__name__)

app.layout =  html.Div(children=[
    
    html.H1(
        children='Latest Covid 19 Update',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }
    ),
    dcc.Tabs(id='tabs-example', value='tab', children=[
        dcc.Tab(label='Total Cases', value='tab-1'),
        dcc.Tab(label='Active Cases', value='tab-2'),
        dcc.Tab(label='Solved Cases', value='tab-3'),
        dcc.Tab(label='Recovery Rate', value='tab-4'),
        dcc.Tab(label='Case Fatality Rate', value='tab-5')
    ]),
    html.Div(id='tabs-example-content'),

    html.Div(
    [
       html.H1(
        children='Latest Covid 19 Update',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }
        ),
        dash_table.DataTable(
            id='table',
        # data import
            data=df.to_dict("rows"),
            columns=[{"name": i, "id": i} for i in df.columns],
        # table interactivity
            editable=True,
            sort_action="native",
            sort_mode="multi",
            row_selectable="multi",
            # style table
            style_table={
                'maxHeight': '50ex',
                'overflowY': 'scroll',
                'width': '100%',
                'minWidth': '100%',
            },
            # style cell
            style_cell={
                'fontFamily': 'Open Sans',
                'textAlign': 'center',
                'height': '30px',
                'padding': '10px 22px',
                'whiteSpace': 'inherit',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis',
            },
            style_cell_conditional=[
                {
                    'if': {'column_id': 'State'},
                    'textAlign': 'left'
                },
            ],
            # style header
            style_header={
                'fontWeight': 'bold',
                'backgroundColor': "#3D9970",
                'fontSize':'16px'
            },
            # style filter
            # style data
            style_data_conditional=[
                {
                    # stripped rows
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                },
                {
                    # highlight one row
                    'if': {'row_index': 44},
                    "backgroundColor": "#3D9970",
                    'color': 'white'
                }
            ],
        ),
   ]),
        html.H1(
        children='Covid 19 Cases in South Africa',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),
    dcc.Graph(
        id='',
        figure=fig_line

        ),
    dcc.Graph(
        id='',
        figure=fig_lin

)
  
])

@app.callback(Output('tabs-example-content', 'children'),
              [Input('tabs-example', 'value')])
def render_content(tab):
    
    if tab == 'tab-1':
        return html.H1(
        children='Confirmed Cases by Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig)
        )
    elif tab == 'tab-2':   
        return html.H1(
        children='Active Cases by Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig1)
        )
           
    elif tab == 'tab-3':   
        return html.H1(
        children='Solved Cases by Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }), dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig2)
        )
    elif tab == 'tab-4':
         return html.H1(
        children='Recovery Rate per country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }), dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig3)
        )
    else:
        return html.H1(
        children='Fatality Rate per Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig4)
        )

# 
@app.callback(Output('tabs-example-content', 'children'),
              [Input('tabs-example', 'value')])
def render_content(tab):
    
    if tab == 'tab-1':
        return html.H1(
        children='Confirmed Cases by Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig)
        )
    elif tab == 'tab-2':   
        return html.H1(
        children='Active Cases by Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig1)
        )
           
    elif tab == 'tab-3':   
        return html.H1(
        children='Solved Cases by Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }), dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig2)
        )
    elif tab == 'tab-4':
         return html.H1(
        children='Recovery Rate per country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }), dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig3)
        )
    else:
        return html.H1(
        children='Fatality Rate per Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }),dcc.Graph(
        id='figure',
        figure=
          go.Figure(fig4)
        )

if __name__ == '__main__':
    app.run_server()

NameError: name 'fig_line' is not defined

In [19]:
# mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjajNlcnh3MzEwMHZtMzNueGw3NWw5ZXF5In0.fk8k06T96Ml9CLGgKmk81w'

# layout = dict(
#     autosize=True,
#     automargin=True,
#     margin=dict(
#         l=30,
#         r=30,
#         b=20,
#         t=40
#     ),
#     hovermode="closest",
#     plot_bgcolor="#F9F9F9",
#     paper_bgcolor="#F9F9F9",
#     legend=dict(font=dict(size=10), orientation='h'),
#     title='Satellite Overview',
#     mapbox=dict(
#         accesstoken=mapbox_access_token,
#         style="light",
#         center=dict(
#             lon=-78.05,
#             lat=42.54
#         ),
#         zoom=7,
#     )
# )

# # Create app layout
# app =  dash.Dash(suppress_callback_exceptions = True)
# app.layout = html.Div(
#     [
#         dcc.Store(id='aggregate_data'),
#         html.Div(
#             [
#                 html.Div(
#                     [
#                         html.H2(
#                             'New York Oil and Gas',

#                         ),
#                         html.H4(
#                             'Production Overview',
#                         )
#                     ],

#                     className='eight columns'
#                 ),
#                 html.Img(
#                     src="https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe.png",
#                     className='four columns',
#                 ),
#                 html.A(
#                     html.Button(
#                         "Learn More",
#                         id="learnMore"

#                     ),
#                     href="https://plot.ly/dash/pricing/",
#                     className="two columns"
#                 )
#             ],
#             id="header",
#             className='row',
#         ),
#         html.Div(
#             [
#                 html.Div(
#                     [
#                         html.P(
#                             'Confirmed Cases by Country:',
#                             className="control_label"
#                         ),dash_table.DataTable(
#                             id='table',
#                         # data import
#                             data=df.to_dict("rows"),
#                             columns=[{"name": i, "id": i} for i in df.columns],
#                         # table interactivity
#                             editable=True,
#                             sort_action="native",
#                             sort_mode="multi",
#                             row_selectable="multi",
#                             # style table
#                             style_table={
#                                 'maxHeight': '50ex',
#                                 'overflowY': 'scroll',
#                                 'width': '100%',
#                                 'minWidth': '100%',
#                             },
#                             # style cell
#                             style_cell={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'center',
#                                 'height': '30px',
#                                 'padding': '10px 22px',
#                                 'whiteSpace': 'inherit',
#                                 'overflow': 'hidden',
#                                 'textOverflow': 'ellipsis',
#                             },
#                             style_cell_conditional=[
#                                 {
#                                     'if': {'column_id': 'State'},
#                                     'textAlign': 'left'
#                                 },
#                             ],
#                             # style header
#                             style_header={
#                                 'fontWeight': 'bold',
#                                 'backgroundColor': "#3D9970",
#                                 'fontSize':'16px'
#                             },
#                             # style filter
#                             # style data
#                             style_data_conditional=[
#                                 {
#                                     # stripped rows
#                                     'if': {'row_index': 'odd'},
#                                     'backgroundColor': 'rgb(248, 248, 248)'
#                                 },
#                                 {
#                                     # highlight one row
#                                     'if': {'row_index': 44},
#                                     "backgroundColor": "#3D9970",
#                                     'color': 'white'
#                                 }
#                             ],
#                         )
                       
#                     ],
#                     className="pretty_container four columns"
#                 ),
#                 html.Div(
#                     [
#                         html.Div(
#                             [
#                                 html.Div(
#                                     [
#                                         html.P("Covid-19 Cases"),
#                                         html.H6(
#                                             id="well_text",
#                                             className="info_text"
#                                         )
#                                     ],
#                                     id="wells",
#                                     className="pretty_container"
#                                 ),

#                                 html.Div(
#                                     [
#                                         html.Div(
#                                             [
#                                                 html.P("Active Cases"),
#                                                 html.H6(
#                                                     id="gasText",
#                                                     className="info_text"
#                                                 )
#                                             ],
#                                             id="gas",
#                                             className="pretty_container"
#                                         ),
#                                         html.Div(
#                                             [
#                                                 html.P("Recovered"),
#                                                 html.H6(
#                                                     id="oilText",
#                                                     className="info_text"
#                                                 )
#                                             ],
#                                             id="oil",
#                                             className="pretty_container"
#                                         ),
#                                         html.Div(
#                                             [
#                                                 html.P("Deaths"),
#                                                 html.H6(
#                                                     id="waterText",
#                                                     className="info_text"
#                                                 )
#                                             ],
#                                             id="water",
#                                             className="pretty_container"
#                                         ),
#                                     ],
#                                     id="tripleContainer",
#                                 )

#                             ],
#                             id="infoContainer",
#                             className="row"
#                         ),
#                         html.Div(
#                             [
#                                 dcc.Tabs(id='tabs-example', value='tab', children=[
#                                 dcc.Tab(label='Total Cases', value='tab-1'),
#                                 dcc.Tab(label='Active Cases', value='tab-2'),
#                                 dcc.Tab(label='Solved Cases', value='tab-3'),
#                                 dcc.Tab(label='Recovery Rate', value='tab-4'),
#                                 dcc.Tab(label='Case Fatality Rate', value='tab-5')
#                             ])
#                                     ,
                                
#                             ],
#                             id="countGraphContainer",
#                             className="pretty_container"
#                         )
#                     ],
#                     id="rightCol",
#                     className="eight columns"
#                 )
#             ],
#             className="row"
#         ),
#         html.Div(
#             [
#                 html.Div(
#                     [
#                          dcc.Graph(
#                             id='main_graph',
#                             figure=fig_line

#                             )
#                     ],
#                     className='pretty_container eight columns',
#                 ),
#                 html.Div(
#                     [
#                          dcc.Graph(
#                             id='',
#                             figure=fig_lin
#                             )
#                     ],
#                     className='pretty_container four columns',
#                 ),
#             ],
#             className='row'
#         ),
#         html.Div(
#             [
#                 html.Div(
#                     [
#                         dcc.Graph(
#                             id='pie_graph',
#                             figure=
#                               go.Figure(fig)
#                             )
#                     ],
#                     className='pretty_container seven columns',
#                 ),
#                 html.Div(
#                     [
#                         dcc.Graph(
#                             id='aggregate_graph',
#                             figure=
#                               go.Figure(fig)
#                             )
#                     ],
#                     className='pretty_container five columns',
#                 ),
#             ],
#             className='row'
#         ),
#     ],
#     id="mainContainer",
#     style={
#         "display": "flex",
#         "flex-direction": "column"
#     }
# )
# @app.callback(Output('tabs-example-content', 'children'),
#               [Input('tabs-example', 'value')])
# def render_content(tab):
    
#     if tab == 'tab-1':
#         return html.H1(
#         children='Confirmed Cases by Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }),dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig)
#         )
#     elif tab == 'tab-2':   
#         return html.H1(
#         children='Active Cases by Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }),dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig1)
#         )
           
#     elif tab == 'tab-3':   
#         return html.H1(
#         children='Solved Cases by Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }), dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig2)
#         )
#     elif tab == 'tab-4':
#          return html.H1(
#         children='Recovery Rate per country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }), dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig3)
#         )
#     else:
#         return html.H1(
#         children='Fatality Rate per Country',
#         style={
#             'textAlign': 'center',
#             'color': 'maroon'
#         }),dcc.Graph(
#         id='figure',
#         figure=
#           go.Figure(fig4)
#         )


# # if __name__ == '__main__':
# #     app.server.run()

In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc

# Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        #header
    
    html.Div([
        html.Div([
            html.Img(src="https://dcassetcdn.com/design_img/3666062/781819/781819_21156990_3666062_e336b60a_image.png",className='four columns')
            
        ], className="four columns"),
        

        html.Div([
            html.H3('Latest Covid 19 Update', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('#StayAtHome', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'right',
                            }),
            
        ], className="four columns")
    ], className="row"),
    
    dcc.Tabs(id='tabs-example', value='tab', children=[
        dcc.Tab(label='Global Map', value='tab-1'),
        dcc.Tab(label='South African cases', value='tab-2'),
        dcc.Tab(label='Analysis', value='tab-3'),
    ]),
    html.Div(id='tabs-example-content')
])

@app.callback(Output('tabs-example-content', 'children'),
               [Input('tabs-example', 'value')])


def render_content(tab):
    '''Display the table and the graph sideby
    side when the respective tab is selected'''

    if tab == 'tab-1':
        return html.Div([
       
            html.H3('band', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            })
         
    ], className="row"),
    
    #VISUALISATIONS
    html.Div([
        html.Div([
            html.H3('visualisation'),
            html.Div([
                html.Div(
                    [
                        html.P("Total confirmed cases"),
                        html.H6(
                            id="well_text",
                            className="info_text"
                        )
                    ],
                    id="wells",
                    className="pretty_container"
                )
            ], className="row"),
            html.Div([
       
            html.H3('Cases by country'),
                dash_table.DataTable(
                        id='table',
                        data=df_tot_cases.to_dict("rows"),
                        columns=[{"name": i, "id": i}
                                 for i in df_tot_cases.columns],
                        style_table=style_table,
                        style_cell=style_cell,
                        style_data_conditional=style_data_conditional,
                        style_header=style_header,
                        style_cell_conditional=style_cell_conditional,
                    )
         
    ], className="row"),
            
        ], className="two columns"),
         html.Div([
            html.H3('visualisation'),
            html.Div([
       
            html.H3('World Map')
         
    ], className="row"),
            
        ], className="six columns"),

        html.Div([
            html.H3('visualisation'),
            
            
            html.Div([
                html.Div([
                    html.H3('Total Recoveries'),
                    
                    ], className="six columns"),

                html.Div([
                    html.H3('Total Deaths'),

                    ], className="six columns"),
    ], className="row")
            
        ], className="four columns"),
    ], className="row"),
    
    #fOOTER
    html.Div([
        html.Div([
            html.H3('Footer'),
            
        ], className="six columns"),

        html.Div([
            html.H3('Footer'),
            
        ], className="six columns"),
    ], className="row")
        

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2020 09:43:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2020 09:43:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2020 09:43:36] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/tshepo/.local/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Apr/2020 09:43:36] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Apr/2020 09:44:06] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/tshepo/.local/lib/python3.6/site-packages/dash/dash.py", line 1

...

127.0.0.1 - - [19/Apr/2020 09:44:17] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Apr/2020 09:44:18] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/tshepo/.local/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Apr/2020 09:44:19] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/tshepo/.local/lib/python3.6/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/tshepo/.local/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Apr/2020 09:44:20] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Apr/2020 09:44:22] "POST /_dash-update-component HTTP/1.1" 200 -
